# Hold-out

In [127]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from rapidfuzz import fuzz, process
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.model_selection import train_test_split

sns.set()

In [128]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")
test_price = pd.read_csv("sample_submission.csv")

In [129]:
test_merge = pd.merge(test, test_price, on='carID', how='left')
test_merge.head()

,carID,Brand,model,year,transmission,mileage,fuelType,tax,mpg,engineSize,paintQuality%,previousOwners,hasDamage,price
0,89856,Hyundai,I30,2022.878006,Automatic,30700.000000,petrol,205.0,41.5,1.6,61.0,3.0,0.0,851000
1,106581,VW,Tiguan,2017.000000,Semi-Auto,-48190.655673,Petrol,150.0,38.2,2.0,60.0,2.0,0.0,514000
2,80886,BMW,2 Series,2016.000000,Automatic,36792.000000,Petrol,125.0,51.4,1.5,94.0,2.0,0.0,323000
3,100174,Opel,Grandland X,2019.000000,Manual,5533.000000,Petrol,145.0,44.1,1.2,77.0,1.0,0.0,921000
4,81376,BMW,1 Series,2019.000000,Semi-Auto,9058.000000,Diesel,150.0,51.4,2.0,45.0,4.0,0.0,620000


In [130]:
X_train_val = train.drop('price', axis=1) 
y_train_val = train['price']
X_test = test_merge.drop('price', axis=1)
y_test = test_merge['price']

In [131]:
X_train, X_val, y_train, y_val = train_test_split(
    X_train_val,
    y_train_val, 
    test_size=0.25, 
    random_state=15
)

In [132]:
print('train:{}% | validation:{}%'.format(round(len(y_train)/len(y_train_val),2),
                                                     round(len(y_val)/len(y_train_val),2),))

train:0.75% | validation:0.25%


In [133]:
X_features = [X_train, X_val,X_test]

## Pre Processing 

In [134]:
for f in X_features:
    f.set_index("carID", inplace=True)
    f["year"]=[int(i) if isinstance(i, float) and not pd.isna(i) else i for i in f["year"]]
    f["year"] = f["year"].astype("Int64")
    f["previousOwners"]=[int(i) if isinstance(i, float) and not pd.isna(i) else i for i in f["previousOwners"]]
    f["previousOwners"] = f["previousOwners"].astype("Int64")

In [135]:
metric_features = ['year', 'mileage','tax','mpg', 'engineSize', 'paintQuality%', 'previousOwners', 'hasDamage']
non_metric_features = X_train.columns.drop(metric_features).to_list()